### Simple SVC model

In [4]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [5]:
def train_test():
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(data, digits.target)
    clf = SVC()
    clf.fit(X_train, y_train)
    test_acc = clf.score(X_test, y_test)
    print("Test accuracy: {:.2f}%".format(test_acc * 100))

train_test()

Test accuracy: 99.56%


### ZenML foundations

In [9]:
from zenml import step
from typing_extensions import Annotated
from pandas import PeriodDtype
from typing import Tuple

In [22]:
@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.2)
    return X_train, X_test, y_train, y_test

In [23]:
@step
def svc_trainer(
    X_train: np.ndarray, 
    y_train: np.ndarray
) -> ClassifierMixin:
    clf = SVC()
    clf.fit(X_train, y_train)
    return clf

In [24]:
@step
def evaluator(
    model: ClassifierMixin,
    X_test: np.ndarray,
    y_test: np.ndarray
) -> float:
    test_acc = model.score(X_test, y_test)
    print("Test accuracy: {:.2f}%".format(test_acc * 100))
    return test_acc

In [25]:
from zenml import pipeline

@pipeline
def digits_pipeline():
    X_train, X_test, y_train, y_test = importer()
    model = svc_trainer(X_train, y_train)
    evaluator(model, X_test, y_test)

In [26]:
digits_svc_pipeline = digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 1).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Step importer has started.
Skipping visualization of numpy array because matplotlib is not installed. To install matplotlib, run pip install matplotlib.
Skipping visualization of numpy array because matplotlib is not installed. To install matplotlib, run pip install matplotlib.
Step importer has finished in 0.247s.
Step svc_trainer has started.
Step svc_trainer has finished in 0.074s.
Step evaluator has started.
Test accuracy: 98.33%
Step evaluator has finished in 0.071s.
Run digits_pipeline-2024_01_08-00_51_22_098253 has finished in 0.492s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


### ZenML project